In [19]:
import numpy as np
import cv2
from cv2 import aruco
import pandas as pd
import matplotlib.colors as mcolors
# import random

In [20]:
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

In [21]:
def imshow(img, window_name="img"):
    cv2.startWindowThread()
    cv2.imshow(window_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [22]:
# 軌跡データ
markers = pd.read_csv("output/test/markers05.csv", header=[0, 1])
frames = len(markers)

# 動画データ
cap = cv2.VideoCapture("src/mov/test05.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))    # 動画のfps
ret, frame = cap.read()
h, w = frame.shape[:2]

ids = pd.Series(markers.columns.levels[0])  # カラムのid列を取得
ids_mask = pd.Series(map(lambda id: str(id).isnumeric(), ids))  # 数値を真、文字列を偽とするマスクを作成
ids = ids.loc[ids_mask] # マスクを使って数値（id）のみを抜き出し

cpxy = ['cpx', 'cpy']
pos = [None, None]
pos_van = [None, None]
pos_lnr = [None, None]
pos_spr = [None, None]
orbits_spr = pd.DataFrame() # スプライン補完した際の軌跡
orbits_lnr = pd.DataFrame() # 線形補間した際の軌跡
orbits_van = pd.DataFrame()  # 元データの軌跡



for id in ids:
    # print("id :", id)
    for i, cp in enumerate(cpxy):
        pos[i] = markers[str(id), cp]
        pos_spr[i] = pos[i]
        pos_van[i] = pos[i]
        pos_lnr[i] = pos[i]
        if (frames - pos[i].isnull().sum()) <= 4:   # 検出できた座標が4以下ならnan（スプライン補完できないため）
            pos_spr[i] = [np.nan]
        else:
            # 通常データ
            pos_van[i] = pos_van[i].astype('Int64')
            # 線形補間
            pos_lnr[i] = pos_lnr[i].interpolate("ffill").interpolate("bfill").astype('int')
            # スプライン補完
            pos_spr[i] = pos_spr[i].interpolate(method="spline", order=2, limit_direction="both").astype(int)
    if ~np.isnan(pos_spr[0][0]) or ~np.isnan(pos_spr[1][0]) :   # 最初の座標がnanでなければorbits_sprに追加(pos_spr[i]が[nan]でなければ全て数値が入っていることは保証されている)
        orbits_van[id] = list(zip(pos_van[0], pos_van[1]))
        orbits_lnr[id] = list(zip(pos_lnr[0], pos_lnr[1]))
        orbits_spr[id] = list(zip(pos_spr[0], pos_spr[1]))

orbits_spr

,0,1,2,3
0,"(352, 92)","(636, 95)","(354, 282)","(627, 281)"
1,"(352, 93)","(638, 96)","(355, 283)","(627, 281)"
2,"(352, 93)","(638, 96)","(355, 283)","(627, 281)"
3,"(352, 93)","(638, 96)","(355, 283)","(627, 281)"
4,"(353, 93)","(638, 96)","(355, 283)","(628, 281)"
...,...,...,...,...
925,"(658, 272)","(401, 280)","(673, 95)","(400, 93)"
926,"(659, 272)","(401, 280)","(673, 95)","(400, 93)"
927,"(659, 272)","(401, 280)","(673, 95)","(401, 93)"
928,"(659, 272)","(402, 280)","(673, 95)","(400, 93)"


In [23]:
# 線のつなぎ方（並び替え）
order_columns = [0,1,3,2]
orbits_van = orbits_van.set_axis(order_columns, axis=1).sort_index(axis=1, ascending=True)
orbits_spr = orbits_spr.set_axis(order_columns, axis=1).sort_index(axis=1, ascending=True)
orbits_lnr = orbits_lnr.set_axis(order_columns, axis=1).sort_index(axis=1, ascending=True)

In [24]:
for f in range(frames):
    ret, img = cap.read()
    if not ret or cv2.waitKey(int(1000 / fps)) == 27:
        break
    # img = np.zeros((h, w, 3))

    # マーカー認識
    corners, ids, rejectedImgPoints = aruco.detectMarkers(img, dictionary)
    # マーカー可視化
    aruco.drawDetectedMarkers(img, corners, ids, (0, 255, 255))

    # 線形補間を描画
    lines_pos_lnr = np.array(list(orbits_lnr.iloc[f]))
    print(lines_pos_lnr)
    cv2.polylines(img, [lines_pos_lnr], True, (255,0,0), 1)

    # スプライン補完を描画
    lines_pos_spr = np.array(list(orbits_spr.iloc[f]))
    cv2.polylines(img, [lines_pos_spr], True, (0,0,255), 1)

    # 通常データを描画（<NA>が入っていれば描画しない）
    lines_pos_van = np.array(list(orbits_van.iloc[f]))
    count_nan = pd.Series(list(map(lambda t: pd.Series(t).isna().sum() ,lines_pos_van))).sum()
    if count_nan == 0:
        cv2.polylines(img, [lines_pos_van], True, (0,255,255), 2)

    cv2.imshow("mov", img)
    # print(f, lines_pos_spr)

cv2.destroyAllWindows()

[[352  93]
 [638  96]
 [627 281]
 [355 283]]
[[352  93]
 [638  96]
 [627 281]
 [355 283]]
[[352  93]
 [638  96]
 [627 281]
 [355 283]]
[[352  93]
 [638  96]
 [627 281]
 [355 283]]
[[353  93]
 [638  96]
 [628 281]
 [355 283]]
[[353  93]
 [638  96]
 [628 281]
 [355 283]]
[[353  93]
 [638  96]
 [628 281]
 [355 283]]
[[353  93]
 [638  96]
 [628 281]
 [355 283]]
[[353  93]
 [638  96]
 [628 281]
 [355 283]]
[[353  93]
 [638  96]
 [628 281]
 [355 283]]
[[353  93]
 [638  96]
 [628 281]
 [356 283]]
[[353  93]
 [638  96]
 [628 281]
 [356 283]]
[[352  93]
 [638  96]
 [628 281]
 [356 283]]
[[352  93]
 [638  96]
 [628 281]
 [356 283]]
[[353  93]
 [638  96]
 [628 281]
 [356 283]]
[[353  93]
 [638  96]
 [628 281]
 [356 283]]
[[353  93]
 [639  96]
 [628 281]
 [356 283]]
[[352  93]
 [639  96]
 [628 281]
 [356 283]]
[[353  93]
 [639  96]
 [628 281]
 [356 283]]
[[353  93]
 [639  96]
 [628 281]
 [356 283]]
[[353  93]
 [639  96]
 [628 281]
 [356 283]]
[[353  93]
 [639  96]
 [628 281]
 [356 283]]
[[353  93]